In [ ]:
!pip install sentence-transformers

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 171.5/171.5 kB 1.5 MB/s eta 0:00:00
  Using cached nvidia_cuda_nvrtc_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (23.7 MB)
  Using cached nvidia_cuda_runtime_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (823 kB)
  Using cached nvidia_cuda_cupti_cu12-12.1.105-py3-none-manylinux1_x86_64.whl (14.1 MB)
  Using cached nvidia_cudnn_cu12-8.9.2.26-py3-none-manylinux1_x86_64.whl (731.7 MB)
  Using cached nvidia_cublas_cu12-12.1.3.1-py3-none-manylinux1_x86_64.whl (410.6 MB)
  Using cached nvidia_cufft_cu12-11.0.2.54-py3-none-manylinux1_x86_64.whl (121.6 MB)
  Using cached nvidia_curand_cu12-10.3.2.106-py3-none-manylinux1_x86_64.whl (56.5 MB)
  Using cached nvidia_cusolver_cu12-11.4.5.107-py3-none-manylinux1_x86_64.whl (124.2 MB)
  Using cached nvidia_cusparse_cu12-12.1.0.106-py3-none-manylinux1_x86_64.whl (196.0 MB)
  Using cached nvidia_nccl_cu12-2.19.3-py3-none-manylinux1_x86_64.whl (166.0 MB)
  Using cached nvidia_nvtx_cu12-12.1.105-py3-none-many

In [ ]:
!pip install ranx

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 99.2/99.2 kB 1.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 337.9/337.9 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 142.5/142.5 kB 14.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 242.1/242.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 43.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 59.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 45.4/45.4 kB 3.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 111.8/111.8 kB 13.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
  Created wh

In [ ]:
# Required imports
from google.colab import drive
from ranx import fuse, Run, Qrels, evaluate
import os
import gzip
import logging
from collections import defaultdict
from sentence_transformers import util
from itertools import combinations


## Load all three `ranking.run` file

In [ ]:
drive.mount('/content/gdrive')
base_path = "./gdrive/MyDrive/cross-encoder-reranker-ir-course-2023/"

Mounted at /content/gdrive


In [ ]:
!mkdir -p $base_path

In [ ]:
# Load ranking.run files for all three evaluated models
base_path = "./gdrive/MyDrive/cross-encoder-reranker-ir-course-2023/finetuned_models/"
file_name = "/ranking.run"
MiniLM_path = "cross-encoder-cross-encoder-ms-marco-MiniLM-L-2-v2-2024-05-04_21-10-05" #@param {type:"string"}
TinyBERT_path = "cross-encoder-cross-encoder-ms-marco-TinyBERT-L-2-v2-2024-05-04_22-28-09" #@param {type:"string"}
distilroBERTa_path = "cross-encoder-distilbert-distilroberta-base-2024-05-05_10-29-29" #@param {type:"string"}

file_list = [base_path + param + file_name for param in [MiniLM_path, TinyBERT_path, distilroBERTa_path]]

## Load the qrels TREC 2019 DL dataset

In [ ]:
!wget https://msmarco.z22.web.core.windows.net/msmarcoranking/queries.tar.gz
!tar -xvzf  queries.tar.gz

--2024-05-08 21:22:37--  https://msmarco.z22.web.core.windows.net/msmarcoranking/queries.tar.gz
Resolving msmarco.z22.web.core.windows.net (msmarco.z22.web.core.windows.net)... 20.150.34.1
Connecting to msmarco.z22.web.core.windows.net (msmarco.z22.web.core.windows.net)|20.150.34.1|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 18882551 (18M) [application/gzip]
Saving to: ‘queries.tar.gz’

queries.tar.gz      100%[===================>]  18.01M  17.8MB/s    in 1.0s    

2024-05-08 21:22:39 (17.8 MB/s) - ‘queries.tar.gz’ saved [18882551/18882551]

queries.dev.tsv
queries.eval.tsv
queries.train.tsv


In [ ]:
"""
This file evaluates CrossEncoder on the TREC 2019 Deep Learning (DL) Track: https://arxiv.org/abs/2003.07820

TREC 2019 DL is based on the corpus of MS Marco. MS Marco provides a sparse annotation, i.e., usually only a single
passage is marked as relevant for a given query. Many other highly relevant passages are not annotated and hence are treated
as an error if a model ranks those high.

TREC DL instead annotated up to 200 passages per query for their relevance to a given query. It is better suited to estimate
the model performance for the task of reranking in Information Retrieval.

Run:
python eval_cross-encoder-trec-dl.py cross-encoder-model-name

"""


data_folder = 'trec2019-data'
os.makedirs(data_folder, exist_ok=True)

#Read test queries
queries = {}
queries_filepath = os.path.join(data_folder, 'msmarco-test2019-queries.tsv.gz')
if not os.path.exists(queries_filepath):
    logging.info("Download "+os.path.basename(queries_filepath))
    util.http_get('https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-test2019-queries.tsv.gz', queries_filepath)

with gzip.open(queries_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        qid, query = line.strip().split("\t")
        queries[qid] = query

#Read which passages are relevant
relevant_docs = defaultdict(lambda: defaultdict(int))
qrels_filepath = os.path.join(data_folder, '2019qrels-pass.txt')

if not os.path.exists(qrels_filepath):
    logging.info("Download "+os.path.basename(qrels_filepath))
    util.http_get('https://trec.nist.gov/data/deep/2019qrels-pass.txt', qrels_filepath)


with open(qrels_filepath) as fIn:
    for line in fIn:
        qid, _, pid, score = line.strip().split()
        score = int(score)
        if score > 0:
            relevant_docs[qid][pid] = score

# Only use queries that have at least one relevant passage
relevant_qid = []
for qid in queries:
    if len(relevant_docs[qid]) > 0:
        relevant_qid.append(qid)


# Read the top 1000 passages that are supposed to be re-ranked
passage_filepath = os.path.join(data_folder, 'msmarco-passagetest2019-top1000.tsv.gz')

if not os.path.exists(passage_filepath):
    logging.info("Download "+os.path.basename(passage_filepath))
    util.http_get('https://msmarco.z22.web.core.windows.net/msmarcoranking/msmarco-passagetest2019-top1000.tsv.gz', passage_filepath)



passage_cand = {}
with gzip.open(passage_filepath, 'rt', encoding='utf8') as fIn:
    for line in fIn:
        qid, pid, query, passage = line.strip().split("\t")
        if qid not in passage_cand:
            passage_cand[qid] = []

        passage_cand[qid].append([pid, passage])

logging.info("Queries: {}".format(len(queries)))

  0%|          | 0.00/4.28k [00:00<?, ?B/s]

  0%|          | 0.00/187k [00:00<?, ?B/s]

  0%|          | 0.00/26.6M [00:00<?, ?B/s]

In [ ]:
qrels_data = {
    qid: {pid: score for pid, score in relevant_docs[qid].items()}
    for qid in relevant_docs if len(relevant_docs[qid]) > 0
}
qrels = Qrels(qrels_data)

## Evalute fusion (ensemble) methods


### Create ranx.fusion `Runs` files

In [ ]:
# Load each run file into a ranx.Run object
runs = [Run.from_file(file, kind="trec") for file in file_list]

### The ComboSUM method
This is score-base method that does not need parameter optimization or training episode either

In [ ]:
combined_run = fuse(
    runs=runs, # List of Runs to fuse
    norm="min-max", # Normalization strategy
    method="sum", # Alias for CombSUM
)

/usr/local/lib/python3.10/dist-packages/ranx/normalization/min_max_norm.py:41: NumbaTypeSafetyWarning: unsafe cast from uint64 to int64. Precision may be lost.
  normalized_run[i] = _min_max_norm(run[q_ids[i]])


In [ ]:
# Define the evaluation metrics
metrics = {
    "ndcg@10": 10,
    "recall@100": 100,
    "map@1000": 1000
}

# Evaluate the combined run
results = evaluate(qrels, combined_run, metrics)

In [ ]:
# Print evaluation results
print(results)

{'ndcg@10': 0.6839713170255659, 'recall@100': 0.5134938081140847, 'map@1000': 0.45423875090422305}


## Task 3: Analyzing most effective ensemble method

The choosen ensemble method: Condorcet  
Runs_list = [MiniLM, TinyBERT, distilroBERTa]  

All combinations: [('MiniLM', 'TinyBERT'), ('MiniLM', 'distilroBERTa'), ('TinyBERT', 'distilroBERTa')]

In [ ]:
candidates = ["MiniLM", "TinyBERT", "distilroBERTa"]

combinations_of_two = list(combinations(candidates, 2))

print(combinations_of_two)

[('MiniLM', 'TinyBERT'), ('MiniLM', 'distilroBERTa'), ('TinyBERT', 'distilroBERTa')]


In [ ]:
combinations_of_two_runs = list(combinations(runs, 2))
metrics = {
    "ndcg@10": 10,
    "recall@100": 100,
    "map@1000": 1000
}

for combination, name in zip(combinations_of_two_runs, combinations_of_two):
  fuse_combination = fuse(
      runs=combination,
      norm="min-max",
      method="condorcet"
  )
  result = evaluate(qrels, fuse_combination, metrics)
  print(name, ": ", result)


('MiniLM', 'TinyBERT') :  {'ndcg@10': 0.6781677820307106, 'recall@100': 0.4817149679386322, 'map@1000': 0.4251215159100591}
('MiniLM', 'distilroBERTa') :  {'ndcg@10': 0.6418291514992519, 'recall@100': 0.45354808538606495, 'map@1000': 0.38408719111728434}
('TinyBERT', 'distilroBERTa') :  {'ndcg@10': 0.6540099288800846, 'recall@100': 0.4237624182441206, 'map@1000': 0.3818146565316562}
